## Imports

In [ ]:
!pip install -q datasets
!pip install llama_index
!pip install llama-index-embeddings-huggingface
!pip install openai
!pip install llama-index-postprocessor-cohere-rerank


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 75.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
from tqdm import tqdm
import os

In [ ]:

from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.vector_stores import VectorStoreQuery
from llama_index.llms.openai import OpenAI as oai


from llama_index.postprocessor.cohere_rerank import CohereRerank

In [ ]:
# OS settings for API keys
os.environ["OPENAI_API_KEY"] = ##
os.environ["PINECONE_API_KEY"] = ##
os.environ["PINECONE_ENV"] = ##
os.environ["COHERE_API_KEY"] = ##

In [ ]:
import numpy as np
import pandas as pd

import openai
from openai import OpenAI

## Experiment with hotpot QA

In [ ]:
from datasets import load_dataset

dataset = load_dataset("hotpot_qa", "distractor", split = "train")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [ ]:
dataset

Dataset({
    features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
    num_rows: 90447
})

In [ ]:
def convert_list_to_sent(para_list):
  """
  Para list is a list of list of sentences
  """

  return [' '.join(x) for x in para_list]

In [ ]:

contexts = []
for i in tqdm(range(3000)):
  contexts += dataset[i]['context']['sentences']

100%|██████████| 3000/3000 [00:00<00:00, 4337.28it/s]


In [ ]:
contexts = [tuple(context) for context in contexts]
contexts = list(set(contexts))
contexts = [list(context) for context in contexts]
contexts = convert_list_to_sent(contexts)
print(contexts[0])

William Jefferson White (December 25, 1831 – April 17, 1913) was a civil rights leader, minister, educator, and journalist in Augusta, Georgia.  He was the founder of Harmony Baptist Church in Augusta in 1869 as well as other churches.  He also was a co-founder of the Augusta Institute in 1867, which would become Morehouse College.  He also helped found Atlanta University and was a trustee of both schools.  He was a founder in 1880 and the managing editor of the "Georgia Baptist", a leading African American newspaper for many years.  He was an outspoken civil rights leader.


In [ ]:
len(contexts)

28356

In [ ]:
print(len(contexts))

28356


## Lets construct some data so we know what is ground truth here

While we get multiple contexts, we test on 100. The idea being retrieval across a variety of vector DB noise

In [ ]:
questions = [dataset[i]['question'] for i in range(100)]
supporting_titles = [dataset[i]['supporting_facts']['title'] for i in range(100)]
answers = [dataset[i]['answer'] for i in range(100)]
level = [dataset[i]['level'] for i in range(100)]

supporting_articles = []
for i in tqdm(range(100)):
  sup_titles = list(set(supporting_titles[i]))
  sup_1 = sup_titles[0]
  sup_2 = sup_titles[1]

  idx_1 = dataset[i]['context']['title'].index(sup_1)
  idx_2 = dataset[i]['context']['title'].index(sup_2)

  supporting_articles.append([' '.join(dataset[i]['context']['sentences'][idx_1]), \
                              ' '.join(dataset[i]['context']['sentences'][idx_2])])




100%|██████████| 100/100 [00:00<00:00, 1331.41it/s]


In [ ]:
questions[9]

supporting_articles[9]

['Mount Panorama Circuit is a motor racing track located in Bathurst, New South Wales, Australia.  It is situated on a hill with the dual official names of Mount Panorama and Wahluu and is best known as the home of the Bathurst 1000 motor race held each October, and the Bathurst 12 Hour event held each February.  The 6.213 km long track is technically a street circuit, and is a public road, with normal speed restrictions, when no racing events are being run, and there are many residences which can only be accessed from the circuit.',
 'The 2013 Liqui Moly Bathurst 12 Hour was an endurance race for a variety of GT and touring car classes, including: GT3 cars, GT4 cars, Group 3E Series Production Cars and Dubai 24 Hour cars.  The event, which was staged at the Mount Panorama Circuit, near Bathurst, in New South Wales, Australia on 10 February 2013, was the eleventh running of the Bathurst 12 Hour.  The race also incorporated the opening round of the 2013 Australian GT Championship.  The 

## Setting up retrieval


Here, we create the indexa and test it out to see how it works



In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core import Document

documents = [Document(text=t) for t in contexts]
index = VectorStoreIndex.from_documents(
    documents,
    llm = oai(temperature = 0, model = 'gpt-3.5-turbo')
    )

In [ ]:
questions[13]

'In which American football game was Malcolm Smith named Most Valuable player?'

In [ ]:
#Example of generation using one question
query_str = "What is the length of the Mount Panorama Circuit, where the 2013 Liqui Moly Bathurst 12 Hour was staged?"
q = index.as_query_engine(similarity_top_k = 4)
query_result = q.query(query_str)

print(query_result.response)

The length of the Mount Panorama Circuit, where the 2013 Liqui Moly Bathurst 12 Hour was staged, is approximately 6.213 kilometers.


In [ ]:
answers[13]

'Super Bowl XLVIII'

In [ ]:
print(query_result.source_nodes[0].text)

The 2017 Liqui Moly Bathurst 12 Hour endurance race for GT and touring car classes, GT3 and GT4 cars was staged on the Mount Panorama Circuit, near Bathurst, in New South Wales, Australia 5 February 2017.  The 15th running of the Bathurst 12 Hour constituted the opening round of the 2017 Intercontinental GT Challenge Series.  For the first time, the winners of the race were awarded the Australian Tourist Trophy.


In [ ]:
supporting_articles[20]

['Longs Drugs is an American chain with approximately 40 drug stores throughout the state of Hawaii.',
 "Warren Bryant was the CEO of Longs Drugs Store Corporation out of California prior to the retail chain's acquisition by CVS/Caremark.  Hired in 2002 to Longs, he was Senior Vice President of The Kroger Company. , a retail grocery chain, from 1999 to 2002.  Prior to that, from 1996 to 1999, he was President and Chief Executive Officer of Dillon Companies, Inc., a retail grocery chain and subsidiary of The Kroger Co.  He is also a director of OfficeMax Incorporated."]

In [ ]:
questions[21]

'Which  American politician did Donahue replaced '

## Vanilla RAG with HotpotQA

In [ ]:
#Evaluating hits @ k [in our case k here is 5]

similarity_top_k = 4
hit_per_q = []
num_hit = 0
num_hit_rel = 0
responses = []
mrr_sum = 0
retrieved_docs = []

engine = index.as_query_engine(similarity_top_k = similarity_top_k)

for i in tqdm(range(0, 100)):
  hit_perc = 0
  q = questions[i]
  hit_flag = False

  query_result = engine.query(q)

  hits = [query_result.source_nodes[i].text for i in range(len(query_result.source_nodes))]

  responses.append(query_result.response)
  retrieved_docs.append(hits)

  for j in range(len(supporting_articles[i])):
    if supporting_articles[i][j] in hits:
      hit_flag = True
      num_hit += 1
      hit_perc += 1

  for j in range(len(hits)):
    if hits[j] in supporting_articles[i]:
      mrr_sum += (1/(j+1))
      break


  if hit_flag:
    num_hit_rel += 1
  hit_per_q += [hit_perc]

100%|██████████| 100/100 [04:57<00:00,  2.97s/it]


In [ ]:
print(sum([hit_per_q[i] == 0 for i in range(100)]))
print(sum([hit_per_q[i] == 1 for i in range(100)]))
print(sum([hit_per_q[i] == 2 for i in range(100)]))

3
39
58


In [ ]:
hard_hits = 0
num_hard = 0
for i in range(100):
  if level[i] == "hard":
    num_hard += 1
    if hit_per_q[i] == 2:
      hard_hits += 1



print(num_hard)
print(hard_hits / num_hard)


16
0.625


In [ ]:
print(num_hit_rel / 100)
print(num_hit / 200)

0.97
0.775


In [ ]:
print(mrr_sum)

92.83333333333334


## Evaluating generation performance

For this, we will make use of an evaluation query run on GPT 3.5 Turbo to see if the responses match

Note this is only the first step. **For the obtained results, we validate each of them manually on a spreadsheet**

In [ ]:
from openai import OpenAI
import openai

In [ ]:
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])


In [ ]:
responses[36]

'David Lee Roth was inducted into the Rock and Roll Hall of Fame.'

In [ ]:
answers[36]

'David Lee Roth'

In [ ]:
questions[36]

'Who was inducted into the Rock and Roll Hall of Fame, David Lee Roth or Cia Berg?'

In [ ]:
def create_prompt_check(idx):

  prompt = "For the question: " + questions[idx] + " Are the following two answers both saying the same thing? " + \
    "Answer 1: " + responses[idx] + ", Answer 2: " + \
    answers[idx] + ". Respond with yes or no"

  return prompt


ans_cor = []

for i in tqdm(range(100)):
  prompt = create_prompt_check(i)



  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0
  )


  if response.choices[0].text.strip().lower().startswith("yes"):
    ans_cor.append(1)
  else:
    ans_cor.append(0)

100%|██████████| 100/100 [00:23<00:00,  4.18it/s]


In [ ]:
#accuracy of the answers

np.sum(ans_cor) / len(ans_cor)

0.68

In [ ]:
#Exporting this evaluation as a dataframe

eval_df = pd.DataFrame({
    'question': questions,
    'ground_truth': answers,
    'rag_response': responses,
    'level': level,
    'correct_or_not': ans_cor
})

eval_df.to_csv("vanilla_hotpot_eval_k4.csv")

In [ ]:
cors = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}
nums = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}

for i in range(len(ans_cor)):
  nums[level[i]] += 1
  if ans_cor[i] == 1 and hit_per_q[i] == 2:
    cors[level[i]] += 1


print(cors['easy'] / nums['easy'])
print(cors['medium'] / nums['medium'])
print(cors['hard'] / nums['hard'])


0.4074074074074074
0.49122807017543857
0.3125


In [ ]:
responses

["Arthur's Magazine was started first.",
 'Delhi',
 'President Richard Nixon',
 "James Henry Miller's wife was English.",
 'Cadmium Chloride is slightly soluble in alcohol, it is also called ethanol.',
 'Jonathan Stark won more Grand Slam titles than Henri Leconte.',
 "Indogrammodes is the genus of moth in the world's seventh-largest country that contains only one species.",
 'Badr Hari was once considered the best kickboxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring.',
 'The Dutch-Belgian television series that "House of Anubis" was based on first aired in September 2006.',
 'The track where the 2013 Liqui Moly Bathurst 12 Hour was staged is the Mount Panorama Circuit.']

In [ ]:
questions[3]

" What nationality was James Henry Miller's wife?"

## Vanilla RAG with re-ranking on HotpotQA

**Note this is a random piece of investigation. This is not considered or reported with the guided retrieval results. This was just done to get some idea of how valid our approaches and implementations are**

In [ ]:
#Outputs 3 reranked documents from 20 initially retrieved documents

cohere_rerank = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=3)
engine_rerank = index.as_query_engine(
    similarity_top_k=20,
    node_postprocessors=[cohere_rerank]
)

In [ ]:
#Evaluating hits @ k [in our case k here is 5]

hit_per_q = []
num_hit = 0
num_hit_rel = 0
responses = []
mrr_sum = 0
retrieved_docs = []

engine = engine_rerank

for i in tqdm(range(0, 100)):
  hit_perc = 0
  q = questions[i]
  hit_flag = False

  query_result = engine.query(q)

  hits = [query_result.source_nodes[i].text for i in range(len(query_result.source_nodes))]

  responses.append(query_result.response)
  retrieved_docs.append(hits)

  for j in range(len(supporting_articles[i])):
    if supporting_articles[i][j] in hits:
      hit_flag = True
      num_hit += 1
      hit_perc += 1

  for j in range(len(hits)):
    if hits[j] in supporting_articles[i]:
      mrr_sum += (1/(j+1))
      break


  if hit_flag:
    num_hit_rel += 1
  hit_per_q += [hit_perc]

100%|██████████| 100/100 [05:43<00:00,  3.43s/it]


In [ ]:
print(sum([hit_per_q[i] == 0 for i in range(100)]))
print(sum([hit_per_q[i] == 1 for i in range(100)]))
print(sum([hit_per_q[i] == 2 for i in range(100)]))

1
39
60


In [ ]:
hard_hits = 0
num_hard = 0
for i in range(10):
  if level[i] == "easy":
    num_hard += 1
    if hit_per_q[i] == 2:
      hard_hits += 1



print(num_hard)
print(hard_hits / num_hard)


1
0.0


In [ ]:
print(num_hit_rel / 100)
print(num_hit / 200)

0.99
0.795


In [ ]:
print(mrr_sum)

97.33333333333333


In [ ]:
ans_cor = []

for i in tqdm(range(100)):
  prompt = create_prompt_check(i)



  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0
  )


  if response.choices[0].text.strip().lower().startswith("yes"):
    ans_cor.append(1)
  else:
    ans_cor.append(0)

100%|██████████| 100/100 [00:38<00:00,  2.58it/s]


In [ ]:
#accuracy of the answers

np.sum(ans_cor) / len(ans_cor)

0.71

In [ ]:
#Exporting this evaluation as a dataframe

eval_df = pd.DataFrame({
    'question': questions,
    'ground_truth': answers,
    'rag_response': responses,
    'level': level,
    'correct_or_not': ans_cor
})

eval_df.to_csv("vanilla_rerank_hotpot_eval.csv")

## Aside: Baseline accuracy evaluation on first 100 based on GPT 3.5

**This is to see accuracy when GPT 3.5 is provided ground truth passages. This basically helps us verify whether retrieval is an issue or it is generation. If accuracy here is very high, it just means we need to retrieve better as given the correct contexts, GPT 3.5 can do a pretty good job with the task.**

In [ ]:
def create_rag_prompt(passage1, passage2, question):

  prompt = """
    We have provided context information below.

    ---------------------
    {passage1}
    ---------------------
    {passage2}
    ---------------------
    Given this information, please answer the question: {question}
  """.format(passage1 = passage1, passage2 = passage2, question = question)

  return prompt

responses = []

for i in tqdm(range(len(questions))):
  passage1 = supporting_articles[i][0]
  passage2 = supporting_articles[i][1]
  question = questions[i]


  prompt = create_rag_prompt(passage1, passage2, question)

  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0,
    max_tokens = 150
  )

  responses.append(response.choices[0].text.strip())

100%|██████████| 100/100 [00:59<00:00,  1.67it/s]


In [ ]:
ans_cor = []

for i in tqdm(range(len(questions))):
  prompt = create_prompt_check(i)

  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0
  )


  if response.choices[0].text.strip().lower().startswith("yes"):
    ans_cor.append(1)
  else:
    ans_cor.append(0)

100%|██████████| 100/100 [00:36<00:00,  2.76it/s]


In [ ]:
#accuracy of the answers

np.sum(ans_cor) / len(ans_cor)

0.83

In [ ]:
#Exporting this evaluation as a dataframe

eval_df = pd.DataFrame({
    'question': questions,
    'ground_truth': answers,
    'rag_response': responses,
    'level': level,
    'correct_or_not': ans_cor
})

eval_df.to_csv("groundtruth_hotpot_eval.csv")

## Experiment: Naive Iteration

In [ ]:
questions[20]

'Which state does the drug stores, of which the CEO is Warren Bryant, are located?'

In [ ]:
def next_q_extraction(ques, passage):

  prompt = """
  Imagine you are answering the question:
  {ques}

  You have the following imformation in the passage below:

  Passage: {passage}

  Based on this, what other information do you need to answer this question. Frame it as a concise one-step question.
  """.format(ques = ques, passage = passage)

  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      temperature = 0,
      max_tokens = 100
    )

  return response.choices[0].text.strip()


def construct_final_prompt(ques, passage_1, passage_2, passage_3, passage_4):

  prompt = """
  You are given the following passages as context:

  Passage 1: {passage_1}
  Passage 2: {passage_2}
  Passage 3: {passage_3}
  Passage 4: {passage_4}

  Based on information in the above contexts, answer the following question: {ques}
  """.format(ques = ques, passage_1 = passage_1, passage_2 = passage_2, \
             passage_3 = passage_3, passage_4 = passage_4)

  return prompt

In [ ]:
similarity_top_k = 4
hit_per_q = []
num_hit = 0
num_hit_rel = 0
responses = []
mrr_sum = 0
engine = index.as_query_engine(similarity_top_k = similarity_top_k)
retrievals = []

for i in tqdm(range(100)):
  hit_perc = 0
  q = questions[i]
  hit_flag = False

  query_result = engine.query(q)

  hits = [query_result.source_nodes[i].text for i in range(len(query_result.source_nodes))]

  first_hit = hits[0]

  #next question extraction:
  # print("first hit")
  # print(first_hit)

  # print("initial answer")
  # print(query_result.response)

  next_q = next_q_extraction(q, first_hit)

  # print("next_q")
  # print(next_q)

  query_result_new = engine.query(next_q)

  new_hits = [query_result_new.source_nodes[i].text for i in range(len(query_result_new.source_nodes))]

  first_new_hit = new_hits[0]

  final_prompt = construct_final_prompt(q, passage_1 = first_hit, passage_2 = hits[1], \
                                        passage_3 = new_hits[0], passage_4 = new_hits[1])

  retrievals.append([hits[0], hits[1], new_hits[0], new_hits[1]])

  # print(final_prompt)

  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=final_prompt,
      temperature = 0,
      max_tokens = 100
    )

  responses.append(response.choices[0].text.strip())

  retrieved_contexts = [hits[0], hits[1], new_hits[0], new_hits[1]]

  for j in range(len(supporting_articles[i])):
    if supporting_articles[i][j] in retrieved_contexts:
      hit_flag = True
      num_hit += 1
      hit_perc += 1

  if hit_flag:
    num_hit_rel += 1

  hit_per_q += [hit_perc]

  for j in range(len(retrieved_contexts)):
    if hits[j] in supporting_articles[i]:
      mrr_sum += (1/(j+1))
      break

100%|██████████| 100/100 [12:38<00:00,  7.59s/it]


In [ ]:
responses[21]

'Sue Donahue'

In [ ]:
print(sum([hit_per_q[i] == 0 for i in range(100)]))
print(sum([hit_per_q[i] == 1 for i in range(100)]))
print(sum([hit_per_q[i] == 2 for i in range(100)]))

5
37
58


In [ ]:
questions[21]

'Which  American politician did Donahue replaced '

In [ ]:
supporting_articles[21]

['Sue Donahue is an American politician.  Donahue was appointed in 2016 to serve in the Arizona State Senate representing the fifth legislative district as a member of the Republican Party.  Donahue replaced Kelli Ward who resigned to run for the United States Senate.  Donahue did not run for re-election in 2016 and was replaced by Sonny Borrelli.',
 'Kelli Ward ("née" Kaznoski; born January 25, 1969) is an American politician and osteopathic physician.  A member of the Republican Party, Ward was elected in 2012 to serve in the Arizona State Senate representing the fifth legislative district.  She was unopposed for election in 2014.  While serving in office, she continued to practice medicine in the emergency departments in Lake Havasu City and Kingman, Arizona.']

In [ ]:
retrievals[21]

['Maurice A. Donahue (September 21, 1918 – January 13, 1999) is an American politician who served as President of the Massachusetts Senate from 1964 to 1971.',
 'Sue Donahue is an American politician.  Donahue was appointed in 2016 to serve in the Arizona State Senate representing the fifth legislative district as a member of the Republican Party.  Donahue replaced Kelli Ward who resigned to run for the United States Senate.  Donahue did not run for re-election in 2016 and was replaced by Sonny Borrelli.',
 'Maurice A. Donahue (September 21, 1918 – January 13, 1999) is an American politician who served as President of the Massachusetts Senate from 1964 to 1971.',
 'Frank J. Donahue (1881–1979) was an American politician who served as the Massachusetts Secretary of the Commonwealth, Chairman of the Massachusetts Democratic State Committee, and as an Associate Justice of the Massachusetts Superior Court.']

In [ ]:
hit_per_q[21]

1

In [ ]:
hard_hits = 0
num_hard = 0
for i in range(100):
  if level[i] == "medium":
    num_hard += 1
    if hit_per_q[i] == 2:
      hard_hits += 1



print(num_hard)
print(hard_hits / num_hard)


57
0.6491228070175439


In [ ]:
print(num_hit_rel / 100)
print(num_hit / 200)

0.95
0.765


In [ ]:
print(mrr_sum)

92.66666666666667


In [ ]:
ans_cor = []

for i in tqdm(range(len(questions))):
  prompt = create_prompt_check(i)



  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0
  )


  if response.choices[0].text.strip().lower().startswith("yes"):
    ans_cor.append(1)
  else:
    ans_cor.append(0)

100%|██████████| 100/100 [00:39<00:00,  2.51it/s]


In [ ]:
#accuracy of the answers

np.sum(ans_cor) / len(ans_cor)

0.63

In [ ]:
#Exporting this evaluation as a dataframe

eval_df = pd.DataFrame({
    'question': questions,
    'ground_truth': answers,
    'rag_response': responses,
    'level': level,
    'correct_or_not': ans_cor
})

eval_df.to_csv("vanilla_hotpot_eval_iter.csv")

In [ ]:
questions[92]

'The Swedish-British entertainment studio executive and film executive producer, who was the Executive producer for a 2016 American action thriller film directed by Babak Najafi, currently serve?'

In [ ]:
cors = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}
nums = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}

for i in range(len(ans_cor)):
  nums[level[i]] += 1
  if ans_cor[i] == 1:
    cors[level[i]] += 1


print(cors['easy'] / nums['easy'])
print(cors['medium'] / nums['medium'])
print(cors['hard'] / nums['hard'])

#dont pay attention to this as this has been manually evaluated

1.0
0.8333333333333334
0.6666666666666666


In [ ]:
nums['medium']

57

#### Analysis of simple policy above

Any continuous policy that is not well thought out results in poorer performance particularly when the original answer is correct. We need a more thought out decomposition based approach that does not loose the performance of the earlier approaches but builds on top of it. Another massive limitation is that the inductive bias is too strong. We have 2 step only here as we knew that it is the case for HotPotQA. What if we didn't know that. We need a robust way to do this so lets think about that.

Lets see if we can use decomposition with an in-context learning based approach to get there.

We will first do this with GPT 3.5 Turbo and then do this using LLAMA-7B


## Experiment: Guided Retrievakl

Lets think how we will do decomposition here.

We need the decomposition to be such that previous answers are inputted into the LM and it uses that somehow.

My first hypothesis is that manual writing of CoTs will lead to best performance in question decomposition so lets try with that

We first create helper functions similar to the other case in order

In [ ]:
def create_reflection_prompt(facts, claim):

  refl_prompt = """
  Based on the given facts, is it reasonable to say that the following claim is true?

  Facts: {facts}
  Claim: {claim}

  Think step by step. End you answer with either yes or no.
  """.format(facts = facts, claim = claim)

  return refl_prompt


def create_claim(question, answer):

  claim_prompt = """
  Convert the following question and its answer into a simple, one-sentence claim.
  Make sure all details from the question are present in the claim.

  Question: {question}
  Answer: {answer}
  """.format(question = question, answer = answer)

  client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])

  claim = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=claim_prompt,
    temperature = 0,
    max_tokens = 100
  )

  return claim.choices[0].text.strip()

def next_q_extraction(ques, passage):

  prompt = """
  You are the given the following information as context:

  -----------
  Context
  {passage}
  -----------

  Based on the information in the context passage below, ask a follow up question
  so that you can better answer the question: {ques}.
  """.format(ques = ques, passage = passage)

  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      temperature = 0,
      max_tokens = 100
    )

  return response.choices[0].text.strip()


def construct_final_prompt(ques, passage_1, passage_2, passage_3, passage_4):

  prompt = """
  You are given the following passages as context

  Passage 1: {passage_1}
  Passage 2: {passage_2}
  Passage 3: {passage_3}
  Passage 4: {passage_4}

  Based on the information in the context passages below, answer the question: {ques}
  """.format(ques = ques, passage_1 = passage_1, passage_2 = passage_2, \
             passage_3 = passage_3, passage_4 = passage_4)

  return prompt


In [ ]:
#Evaluation block
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
similarity_top_k = 4
hit_per_q = []
num_hit = 0
num_hit_rel = 0
responses = []
mrr_sum = 0
engine = index.as_query_engine(similarity_top_k = similarity_top_k,
                               llm = oai(temperature = 0, model = "gpt-3.5-turbo-instruct"))

# retrievals = []

for i in tqdm(range(20, 100)):
  print("Iteration " + str(i))
  hit_perc = 0
  q = questions[i]
  hit_flag = False

  # query_result = engine.query(q)
  query_result = engine.query(q)

  hits = [query_result.source_nodes[i].text for i in range(len(query_result.source_nodes))]

  facts_string = ""

  for j in range(2):
    facts_string += "Fact " + str(j+1) + ": " + hits[j] + "\n"

  claim = create_claim(q, query_result.response)
  print(f"Question: {q}")
  print(f"Response: {query_result.response}")
  print(f"Claim: {claim}")

  print(f"Facts string: {facts_string}")

  refl_prompt = create_reflection_prompt(facts_string, query_result.response)

  reflection = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=refl_prompt,
    temperature = 0,
    max_tokens = 100
  )

  print(reflection.choices[0].text.strip())

  if reflection.choices[0].text.strip().lower().split(' ')[0].startswith("no"):

    print("follow-up question chosen")

    next_q = next_q_extraction(q, facts_string)

    print(f"Next question: {next_q}")

    # query_result_new = engine.query(next_q)
    query_result_new = engine.query(next_q)

    new_hits = [query_result_new.source_nodes[i].text for i in range(len(query_result_new.source_nodes))]

    final_prompt = construct_final_prompt(q, passage_1 = hits[0], passage_2 = hits[1], \
                                          passage_3 = new_hits[0], passage_4 = new_hits[1])


    retrievals.append([hits[0], hits[1], new_hits[0], new_hits[1]])

    response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=final_prompt,
      temperature = 0,
      max_tokens = 100
    ).choices[0].text.strip()

    retrieved_contexts = [hits[0], hits[1], new_hits[0], new_hits[1]]


  else:

    response = query_result.response

    retrievals.append(hits)

    retrieved_contexts = hits


  responses.append(response)


  for j in range(len(supporting_articles[i])):
    if supporting_articles[i][j] in retrieved_contexts:
      hit_flag = True
      num_hit += 1
      hit_perc += 1

  if hit_flag:
    num_hit_rel += 1

  hit_per_q += [hit_perc]

  for j in range(len(retrieved_contexts)):
    if hits[j] in supporting_articles[i]:
      mrr_sum += (1/(j+1))
      break

  0%|          | 0/80 [00:00<?, ?it/s]

Iteration 20
Question: Which state does the drug stores, of which the CEO is Warren Bryant, are located?
Response: The drug stores are located in California, Arkansas, Oklahoma, Mississippi, Missouri, Tennessee, and Michigan.
Claim: The drug stores, of which the CEO is Warren Bryant, are located in California, Arkansas, Oklahoma, Mississippi, Missouri, Tennessee, and Michigan.
Facts string: Fact 1: Warren Bryant was the CEO of Longs Drugs Store Corporation out of California prior to the retail chain's acquisition by CVS/Caremark.  Hired in 2002 to Longs, he was Senior Vice President of The Kroger Company. , a retail grocery chain, from 1999 to 2002.  Prior to that, from 1996 to 1999, he was President and Chief Executive Officer of Dillon Companies, Inc., a retail grocery chain and subsidiary of The Kroger Co.  He is also a director of OfficeMax Incorporated.
Fact 2: USA Drug is a convenience store and pharmacy chain in the United States that operates more than 160 stores in Arkansas, O

  1%|▏         | 1/80 [00:08<11:23,  8.65s/it]

Iteration 21
Question: Which  American politician did Donahue replaced 
Response:  Sue Donahue replaced Kelli Ward.
Claim: Sue Donahue replaced Kelli Ward as an American politician.
Facts string: Fact 1: Maurice A. Donahue (September 21, 1918 – January 13, 1999) is an American politician who served as President of the Massachusetts Senate from 1964 to 1971.
Fact 2: Sue Donahue is an American politician.  Donahue was appointed in 2016 to serve in the Arizona State Senate representing the fifth legislative district as a member of the Republican Party.  Donahue replaced Kelli Ward who resigned to run for the United States Senate.  Donahue did not run for re-election in 2016 and was replaced by Sonny Borrelli.

No
follow-up question chosen
Next question: Who did Donahue replace in the Arizona State Senate in 2016?


  2%|▎         | 2/80 [00:16<10:33,  8.12s/it]

...

 98%|█████████▊| 78/80 [10:11<00:37, 18.51s/it]

Iteration 98
Question: The Thoen Stone is on display at a museum in what county?
Response: 
The Thoen Stone is on display at the Adams Museum & House in Deadwood, South Dakota, which is located in Lawrence County.
Claim: The Thoen Stone is on display at the Adams Museum & House in Deadwood, South Dakota, which is located in Lawrence County.
Facts string: Fact 1: The Thoen Stone is a sandstone slab dated 1834 that was discovered in the Black Hills of South Dakota by Louis Thoen in 1887.  The discovery of the stone called into question the first discovery of gold and the history of gold mining in the Black Hills; it would mean that gold was discovered in the Black Hills 40 years before the Custer Expedition of 1874 and the subsequent Black Hills Gold Rush.  It is currently on display at the Adams Museum & House in Deadwood, South Dakota.
Fact 2: The Kings County Museum is located in Hampton, New Brunswick and is run by the Kings County Historical and Archival Society, housed in the Cente

 99%|█████████▉| 79/80 [10:15<00:14, 14.24s/it]

Yes
Iteration 99
Question: Is Children's National Medical Center or MedStar Washington Hospital Center the largest private hospital in Washington, D.C.?
Response: MedStar Washington Hospital Center
Claim: MedStar Washington Hospital Center is the largest private hospital in Washington, D.C.
Facts string: Fact 1: MedStar Washington Hospital Center is the largest private hospital in Washington, D.C. A member of MedStar Health, the not-for-profit Hospital Center is licensed for 926 beds.  Health services in primary, secondary and tertiary care are offered to adult and neonatal patients.  It also serves as a teaching hospital for Georgetown University School of Medicine.
Fact 2: Children’s National Medical Center (formerly DC Children’s Hospital) is ranked among the top 10 children’s hospitals in the country by "U.S. News & World Report."  Located just north of the McMillan Reservoir and Howard University, it shares grounds with Washington Hospital Center, National Rehabilitation Hospital,

100%|██████████| 80/80 [10:23<00:00,  7.80s/it]


In [ ]:
responses

["Arthur's Magazine was started first.",
 'Delhi',
 'President Richard Nixon',
 "It is not possible to determine the nationality of James Henry Miller's wife based on the given context passages. The passages only mention James Henry Miller and his occupation as a footballer, but do not provide any information about his wife's nationality.",
 'Cadmium chloride is slightly soluble in alcohol.',
 'Jonathan Stark won more Grand Slam titles.',
 "The genus of moth in the world's seventh-largest country that contains only one species is Indogrammodes.",
 'Badr Hari',
 'The Dutch-Belgian television series that "House of Anubis" was based on first aired in September 2006.',
 'The length of the track where the 2013 Liqui Moly Bathurst 12 Hour was staged is not explicitly stated in the context passages. However, it can be inferred that the event was staged at the Mount Panorama Circuit, near Bathurst, in New South Wales, Australia. According to the official website of the Mount Panorama Circuit, 

In [ ]:
supporting_articles[11]

['Gunmen from Laredo is a 1959 American western film produced and directed by Wallace MacDonald, which stars Robert Knapp, Maureen Hingert, and Walter Coy.',
 'Walter Darwin Coy (January 31, 1909 – December 11, 1974) was an American stage, radio, film, and, principally, television actor, originally from Great Falls, Montana.  He was best known for narrating the NBC western anthology series, "Frontier", which aired early Sunday evenings in the 1955–1956 season.']

In [ ]:
responses

[]

In [ ]:
retrievals[len(retrievals) - 1]

['Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany.  The line-up consists of Martin Caba, Christoph Günther and Christoph Leipold playing Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties.  The expression "Säuwäntzt" means pork belly and refers also to untidy or unruly children and youth.',
 'Born in Vienna, Schörg first trained for the profession of typist.  In 1937, at the age of 23, she won a beauty contest and became " Queen of Danube ".  That same year she made her stage debut in Ostrava at the Antonín Dvořák Theatre in the world premiere of Ödön von Horváth\'s play "Der jüngste Tag".  She spent the next two years performing at that theatre, and at theatres in Karlovy Vary, Ostrava, Marienbad, and Vienna (Theater in der Josefstadt).  In 1939 she joined the opera house in Aussig (now Usti nad Labem) where she became popular as a performer in operettas.  She made her professional opera debut at that h

In [ ]:
questions[:15]

["Which magazine was started first Arthur's Magazine or First for Women?",
 'The Oberoi family is part of a hotel company that has a head office in what city?',
 'Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?',
 " What nationality was James Henry Miller's wife?",
 'Cadmium Chloride is slightly soluble in this chemical, it is also called what?',
 'Which tennis player won more Grand Slam titles, Henri Leconte or Jonathan Stark?',
 "Which genus of moth in the world's seventh-largest country contains only one species?",
 'Who was once considered the best kick boxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring.',
 'The Dutch-Belgian television series that "House of Anubis" was based on first aired in what year?',
 'What is the length of the track where the 2013 Liqui Moly Bathurst 12 Ho

In [ ]:
responses

['First for Women was started first.',
 'The Oberoi family is part of a hotel company that has a head office in Delhi.',
 'President Richard Nixon',
 'Australian',
 'Cadmium chloride is also known as CdCl•5HO.',
 'Jonathan Stark won more Grand Slam titles than Henri Leconte.',
 "Indogrammodes is the genus of moth in the world's seventh-largest country that contains only one species.",
 'Badr Hari was once considered the best kickboxer in the world, however he has been involved in a number of controversies relating to his "unsportsmanlike conducts" in the sport and crimes of violence outside of the ring.',
 'The Dutch-Belgian television series that "House of Anubis" was based on first aired in September 2006.',
 'The length of the track where the 2013 Liqui Moly Bathurst 12 Hour was staged is approximately 6.213 kilometers.',
 'El-P',
 'The narrator of "Frontier" is not mentioned in either passage, so it is not possible to determine who starred in Gunmen from Laredo based on these passa

In [ ]:
facts_string = ""
for j in range(len(retrievals[0])):
    facts_string += "Context " + str(j+1) + ": " + retrievals[0][j] + "\n"

print(facts_string)

Context 1: First for Women is a woman's magazine published by Bauer Media Group in the USA.  The magazine was started in 1989.  It is based in Englewood Cliffs, New Jersey.  In 2011 the circulation of the magazine was 1,310,696 copies.
Context 2: Arthur's Home Magazine (1852-ca.1898) or Ladies' Home Magazine was an American periodical published in Philadelphia by Timothy Shay Arthur.  Editors Arthur and Virginia Francis Townsend selected writing and illustrations intended to appeal to female readers.  Among the contributors: Mary Tyler Peabody Mann and Kate Sutherland.  In its early years the monthly comprised a selection of articles originally published in Arthur's weekly "Home Gazette."  Its nonfiction stories contained occasional factual inaccuracies for the sake of a good read.  A contemporary review judged it "gotten up in good taste and well; and is in nothing overdone.  Even its fashion plates are not quite such extravagant caricatures of rag-baby work as are usually met with in

In [ ]:
p = create_reflection_prompt(facts_string, )

print(p)


  Given the following contexts, tell me if the answer to the question below is justified by the contexts.
  Be careful and pay attention to specific details in the contexts.

  -------------------
  Contexts:
  Context 1: "Human Error" is the twenty-fourth episode and season finale of the third season of "House" and the seventieth episode overall.
Context 2: Being Human is a supernatural drama television series broadcast on Syfy in the United States and on Space in Canada.  It was produced by Muse in Montreal as a remake of the BBC show of the same name.
Context 3: Being Human is a British supernatural drama series created and written by Toby Whithouse for Touchpaper Television.  The fifth and final series began airing on BBC Three on 3 February 2013 and continued until mid-March 2013.  The series follows the lives of a new ghost, vampire and werewolf trio living together and attempting to lead a normal life, and blends a mixture of flatshare comedy and horror drama.
Context 4: Being 

In [ ]:
reflection = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=p,
    temperature = 0.9,
    max_tokens = 100
  )

print(reflection.choices[0].text.strip())

Yes. In context 1, it is mentioned that the episode "Human Error" is the seventieth episode overall, implying that it is part of a TV show with previous episodes. In context 2, it is mentioned that Being Human is a supernatural drama television series broadcast on Syfy in the United States and on Space in Canada. This aligns with the given answer that the TV show in question aired on Syfy. Additionally, in context 4, it is mentioned that the


In [ ]:
retrievals[17]

['"Human Error" is the twenty-fourth episode and season finale of the third season of "House" and the seventieth episode overall.',
 'Being Human is a supernatural drama television series broadcast on Syfy in the United States and on Space in Canada.  It was produced by Muse in Montreal as a remake of the BBC show of the same name.',
 'Being Human is a British supernatural drama series created and written by Toby Whithouse for Touchpaper Television.  The fifth and final series began airing on BBC Three on 3 February 2013 and continued until mid-March 2013.  The series follows the lives of a new ghost, vampire and werewolf trio living together and attempting to lead a normal life, and blends a mixture of flatshare comedy and horror drama.',
 'Being Human is a supernatural drama television series developed for North American television by Jeremy Carver and Anna Fricke, based upon the British series of the same name created by Toby Whithouse.  The series premiered on Syfy and Space Channe

In [ ]:
retrievals[12]

['Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany.  The line-up consists of Martin Caba, Christoph Günther and Christoph Leipold playing Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties.  The expression "Säuwäntzt" means pork belly and refers also to untidy or unruly children and youth.',
 'Born in Vienna, Schörg first trained for the profession of typist.  In 1937, at the age of 23, she won a beauty contest and became " Queen of Danube ".  That same year she made her stage debut in Ostrava at the Antonín Dvořák Theatre in the world premiere of Ödön von Horváth\'s play "Der jüngste Tag".  She spent the next two years performing at that theatre, and at theatres in Karlovy Vary, Ostrava, Marienbad, and Vienna (Theater in der Josefstadt).  In 1939 she joined the opera house in Aussig (now Usti nad Labem) where she became popular as a performer in operettas.  She made her professional opera debut at that h

In [ ]:
responses[12]

'The form of music played by Die Rhöner Säuwäntzt, Skiffle-Blues, originated in Eichenzell-Lütter in Hessen, Germany.'

In [ ]:
hits = retrievals[12]

facts_string = ""

for j in range(len(hits)):
  facts_string += "Fact " + str(j+1) + ": " + hits[j] + "\n"

claim = create_claim(q, responses[12])
print(f"Question: {questions[12]}")
print(f"Response: {responses[12]}")
print(f"Claim: {claim}")

refl_prompt = create_reflection_prompt(facts_string, claim)
print(refl_prompt)

reflection = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=refl_prompt,
  temperature = 0,
  max_tokens = 100
)

print(reflection.choices[0].text.strip())


Question: Where did the form of music played by Die Rhöner Säuwäntzt originate?
Response: The form of music played by Die Rhöner Säuwäntzt, Skiffle-Blues, originated in Eichenzell-Lütter in Hessen, Germany.
Claim: The form of music known as Skiffle-Blues originated in Eichenzell-Lütter, Germany.

  Do the following facts imply the stated claim?

  Facts: Fact 1: Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany.  The line-up consists of Martin Caba, Christoph Günther and Christoph Leipold playing Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties.  The expression "Säuwäntzt" means pork belly and refers also to untidy or unruly children and youth.
Fact 2: Born in Vienna, Schörg first trained for the profession of typist.  In 1937, at the age of 23, she won a beauty contest and became " Queen of Danube ".  That same year she made her stage debut in Ostrava at the Antonín Dvořák Theatre in the world premiere 

In [ ]:
print(responses)

["Arthur's Magazine was started before First for Women.", 'Delhi', 'President Richard Nixon', 'Australian', 'Cadmium chloride is also known as CdCl•HO and CdCl•5HO.', 'Jonathan Stark won more Grand Slam titles than Henri Leconte.', "Indogrammodes is the genus of moth in the world's seventh-largest country that contains only one species.", 'Badr Hari', 'The Dutch-Belgian television series that "House of Anubis" was based on first aired in September 2006.', 'The length of the track where the 2013 Liqui Moly Bathurst 12 Hour was staged is 6.213 km.']


In [ ]:
responses[21]

'Sue Donahue'

In [ ]:
print(sum([hit_per_q[i] == 0 for i in range(100)]))
print(sum([hit_per_q[i] == 1 for i in range(100)]))
print(sum([hit_per_q[i] == 2 for i in range(100)]))

4
43
53


In [ ]:
questions[21]

'Which  American politician did Donahue replaced '

In [ ]:
supporting_articles[21]

['Sue Donahue is an American politician.  Donahue was appointed in 2016 to serve in the Arizona State Senate representing the fifth legislative district as a member of the Republican Party.  Donahue replaced Kelli Ward who resigned to run for the United States Senate.  Donahue did not run for re-election in 2016 and was replaced by Sonny Borrelli.',
 'Kelli Ward ("née" Kaznoski; born January 25, 1969) is an American politician and osteopathic physician.  A member of the Republican Party, Ward was elected in 2012 to serve in the Arizona State Senate representing the fifth legislative district.  She was unopposed for election in 2014.  While serving in office, she continued to practice medicine in the emergency departments in Lake Havasu City and Kingman, Arizona.']

In [ ]:
retrievals[21]

['The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group.',
 'The Oberoi Group is a hotel company with its head office in Delhi.  Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands.',
 "Rai Bahadur Mohan Singh Oberoi (15 August 1898\xa0– 3 May 2002) was an Indian hotelier, the founder and chairman of Oberoi Hotels & Resorts, India's second-largest hotel company, with 35 hotels in India, Sri Lanka, Nepal, Egypt, Australia and Hungary.",
 'Hotel Principe di Savoia, or simply referred to as The Principe, is a five-star hotel in Milan, Italy.  The hotel has long become one of Milan\'s Landmarks.  Founded by the name of "Principe e Savoia" in 1927, a time when Milan had already established its position as financial center of the country, it soon became a home for businessmen.  With time, celebrities o

In [ ]:
hit_per_q[21]

1

In [ ]:
hard_hits = 0
num_hard = 0
for i in range(len(responses)):
  if level[i] == "easy":
    num_hard += 1
    if hit_per_q[i] == 2:
      hard_hits += 1



print(num_hard)
print(hard_hits / num_hard)


27
0.2962962962962963


In [ ]:
len(responses)

100

In [ ]:
hard_hits = 0
for i in range(len(responses)):
  if hit_per_q[i] == 2:
    hard_hits += 1

hard_hits/100

0.53

In [ ]:
print(num_hit_rel / 100)
print(num_hit / 200)

0.96
0.745


In [ ]:
print(mrr_sum)

8.5


In [ ]:
ans_cor = []

for i in tqdm(range(len(responses))):
  prompt = create_prompt_check(i)



  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0
  )


  if response.choices[0].text.strip().lower().startswith("yes"):
    ans_cor.append(1)
  else:
    ans_cor.append(0)

100%|██████████| 100/100 [00:36<00:00,  2.72it/s]


In [ ]:
#accuracy of the answers

np.sum(ans_cor) / len(ans_cor)

0.68

In [ ]:
#Exporting this evaluation as a dataframe

eval_df = pd.DataFrame({
    'question': questions,
    'ground_truth': answers,
    'rag_response': responses,
    'level': level,
    'correct_or_not': ans_cor
})

eval_df.to_csv("vanilla_hotpot_eval_refl_2.csv")

In [ ]:
questions[92]

'The Swedish-British entertainment studio executive and film executive producer, who was the Executive producer for a 2016 American action thriller film directed by Babak Najafi, currently serve?'

In [ ]:
cors = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}
nums = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}

for i in range(len(ans_cor)):
  nums[level[i]] += 1
  if ans_cor[i] == 1:
    cors[level[i]] += 1


print(cors['easy'] / nums['easy'])
print(cors['medium'] / nums['medium'])
print(cors['hard'] / nums['hard'])

#dont pay attention to this as this has been manually evaluated

0.8518518518518519
0.6666666666666666
0.375


## Improved Reflective / Corrective-RAG

Here, we will improve our implementation using a re-ranker alongside the reflective / corrective RAG system

We first create helper functions similar to the other case in order

In [ ]:
#Evaluation block
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
similarity_top_k = 4
# hit_per_q = []
# num_hit = 0
# num_hit_rel = 0
# responses = []
# mrr_sum = 0

cohere_rerank = CohereRerank(api_key=os.environ["COHERE_API_KEY"], top_n=3)

engine_rerank = index.as_query_engine(
    similarity_top_k=20,
    llm = oai(temperature = 0, model = "gpt-3.5-turbo-instruct"),
    node_postprocessors=[cohere_rerank]
)

# retrievals = []

for i in tqdm(range(59, 100)):
  print("Iteration " + str(i))
  hit_perc = 0
  q = questions[i]
  hit_flag = False

  # query_result = engine.query(q)
  query_result = engine_rerank.query(q)

  hits = [query_result.source_nodes[i].text for i in range(len(query_result.source_nodes))]

  facts_string = ""

  for j in range(2):
    facts_string += "Fact " + str(j+1) + ": " + hits[j] + "\n"

  claim = create_claim(q, query_result.response)
  print(f"Question: {q}")
  print(f"Response: {query_result.response}")
  print(f"Claim: {claim}")

  print(f"Facts string: {facts_string}")

  refl_prompt = create_reflection_prompt(facts_string, query_result.response)

  reflection = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=refl_prompt,
    temperature = 0,
    max_tokens = 100
  )

  print(reflection.choices[0].text.strip())

  if reflection.choices[0].text.strip().lower().split(' ')[0].startswith("no"):

    print("follow-up question chosen")

    next_q = next_q_extraction(q, facts_string)

    print(f"Next question: {next_q}")

    # query_result_new = engine.query(next_q)
    query_result_new = engine_rerank.query(next_q)

    new_hits = [query_result_new.source_nodes[i].text for i in range(len(query_result_new.source_nodes))]

    final_prompt = construct_final_prompt(q, passage_1 = hits[0], passage_2 = hits[1], \
                                          passage_3 = new_hits[0], passage_4 = new_hits[1])


    retrievals.append([hits[0], hits[1], new_hits[0], new_hits[1]])

    response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=final_prompt,
      temperature = 0,
      max_tokens = 100
    ).choices[0].text.strip()

    retrieved_contexts = [hits[0], hits[1], new_hits[0], new_hits[1]]


  else:

    response = query_result.response

    retrievals.append(hits)

    retrieved_contexts = hits


  responses.append(response)


  for j in range(len(supporting_articles[i])):
    if supporting_articles[i][j] in retrieved_contexts:
      hit_flag = True
      num_hit += 1
      hit_perc += 1

  if hit_flag:
    num_hit_rel += 1

  hit_per_q += [hit_perc]

  # for j in range(len(retrieved_contexts)):
  #   if hits[j] in supporting_articles[i]:
  #     mrr_sum += (1/(j+1))
  #     break

  0%|          | 0/41 [00:00<?, ?it/s]

Iteration 59
Question: An edited version of "Just the Two of Us" reached number two on the "Billboard" Hot 100 behind a song written and composed by Donna Weiss and Jackie DeShannon which spend how many weeks at No. 1 on the "Billboard" hot 100?
Response: 9 weeks
Claim: The song "Just the Two of Us" reached number two on the "Billboard" Hot 100, behind a song written and composed by Donna Weiss and Jackie DeShannon, which spent 9 weeks at number one on the "Billboard" Hot 100.
Facts string: Fact 1: "Just the Two of Us" is a 1981 R&B single written by Bill Withers, William Salter and Ralph MacDonald, which was recorded by Grover Washington Jr. and Bill Withers.  The song originally appeared on Washington's 1980 album "Winelight".  An edited version reached number two on the "Billboard" Hot 100, staying there for 3 weeks, behind "Morning Train (9 to 5)" by Sheena Easton and "Bette Davis Eyes" by Kim Carnes.  The song won a Grammy Award for Best R&B Song.  Bill Withers included the edited

  2%|▏         | 1/41 [00:09<06:09,  9.23s/it]

Iteration 60
Question: Which band has more members, Saint Motel or Curve?
Response: Saint Motel
Claim: Saint Motel has more members than Curve.
Facts string: Fact 1: Saint Motel is an American indie pop band from Los Angeles, whose music has been described as everything from "dream pop" to "indie prog".  The band consists of A/J Jackson (lead vocals, guitar, piano), Aaron Sharp (lead guitar), Dak Lerdamornpong (bass), and Greg Erwin (drums).
Fact 2: Curve were an English alternative rock and electronica duo from London, formed in 1990 and split in 2005.  The band consisted of Toni Halliday (vocals, occasionally guitar) and Dean Garcia (bass, guitar, drums, programming).  Halliday wrote also the lyrics of their songs and they both contributed to songwriting.  An important collaborator was the producer Alan Moulder, who helped them to shape their blend of heavy beats and densely layered guitar tracks set against Halliday's vocals.

No
follow-up question chosen
Next question: What are the

  5%|▍         | 2/41 [00:20<06:40, 10.27s/it]

...

 95%|█████████▌| 39/41 [05:01<00:17,  8.81s/it]

Iteration 98
Question: The Thoen Stone is on display at a museum in what county?
Response: The Thoen Stone is on display at the Adams Museum & House in Deadwood, South Dakota.
Claim: The Thoen Stone is on display at the Adams Museum & House in Deadwood, South Dakota, which is located in Adams County.
Facts string: Fact 1: The Thoen Stone is a sandstone slab dated 1834 that was discovered in the Black Hills of South Dakota by Louis Thoen in 1887.  The discovery of the stone called into question the first discovery of gold and the history of gold mining in the Black Hills; it would mean that gold was discovered in the Black Hills 40 years before the Custer Expedition of 1874 and the subsequent Black Hills Gold Rush.  It is currently on display at the Adams Museum & House in Deadwood, South Dakota.
Fact 2: The Clay County Historical Society Museum is located in Green Cove Springs, Clay County, Florida.  It is located in a former trail depot.  Exhibits include railroad memorabilia, a count

 98%|█████████▊| 40/41 [05:06<00:07,  7.53s/it]

Yes
Iteration 99
Question: Is Children's National Medical Center or MedStar Washington Hospital Center the largest private hospital in Washington, D.C.?
Response: MedStar Washington Hospital Center
Claim: MedStar Washington Hospital Center is the largest private hospital in Washington, D.C.
Facts string: Fact 1: MedStar Washington Hospital Center is the largest private hospital in Washington, D.C. A member of MedStar Health, the not-for-profit Hospital Center is licensed for 926 beds.  Health services in primary, secondary and tertiary care are offered to adult and neonatal patients.  It also serves as a teaching hospital for Georgetown University School of Medicine.
Fact 2: Children’s National Medical Center (formerly DC Children’s Hospital) is ranked among the top 10 children’s hospitals in the country by "U.S. News & World Report."  Located just north of the McMillan Reservoir and Howard University, it shares grounds with Washington Hospital Center, National Rehabilitation Hospital,

100%|██████████| 41/41 [05:14<00:00,  7.68s/it]


In [ ]:
len(responses)

100

In [ ]:
responses[12]

'The form of music played by Die Rhöner Säuwäntzt, Skiffle-Blues, originated in Eichenzell-Lütter in Hessen, Germany.'

In [ ]:
hits = retrievals[12]

facts_string = ""

for j in range(len(hits)):
  facts_string += "Fact " + str(j+1) + ": " + hits[j] + "\n"

claim = create_claim(q, responses[12])
print(f"Question: {questions[12]}")
print(f"Response: {responses[12]}")
print(f"Claim: {claim}")

refl_prompt = create_reflection_prompt(facts_string, claim)
print(refl_prompt)

reflection = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt=refl_prompt,
  temperature = 0,
  max_tokens = 100
)

print(reflection.choices[0].text.strip())


Question: Where did the form of music played by Die Rhöner Säuwäntzt originate?
Response: The form of music played by Die Rhöner Säuwäntzt, Skiffle-Blues, originated in Eichenzell-Lütter in Hessen, Germany.
Claim: The form of music known as Skiffle-Blues originated in Eichenzell-Lütter, Germany.

  Do the following facts imply the stated claim?

  Facts: Fact 1: Die Rhöner Säuwäntzt are a Skiffle-Bluesband from Eichenzell-Lütter in Hessen, Germany.  The line-up consists of Martin Caba, Christoph Günther and Christoph Leipold playing Skiffle-Blues with lyrics based on Rhön Mountains dialect and other Hessian dialects varieties.  The expression "Säuwäntzt" means pork belly and refers also to untidy or unruly children and youth.
Fact 2: Born in Vienna, Schörg first trained for the profession of typist.  In 1937, at the age of 23, she won a beauty contest and became " Queen of Danube ".  That same year she made her stage debut in Ostrava at the Antonín Dvořák Theatre in the world premiere 

In [ ]:
print(responses)

["Arthur's Magazine was started before First for Women.", 'Delhi', 'President Richard Nixon', 'Australian', 'Cadmium chloride is also known as CdCl•HO and CdCl•5HO.', 'Jonathan Stark won more Grand Slam titles than Henri Leconte.', "Indogrammodes is the genus of moth in the world's seventh-largest country that contains only one species.", 'Badr Hari', 'The Dutch-Belgian television series that "House of Anubis" was based on first aired in September 2006.', 'The length of the track where the 2013 Liqui Moly Bathurst 12 Hour was staged is 6.213 km.']


In [ ]:
responses[21]

'Sue Donahue'

In [ ]:
print(sum([hit_per_q[i] == 0 for i in range(100)]))
print(sum([hit_per_q[i] == 1 for i in range(100)]))
print(sum([hit_per_q[i] == 2 for i in range(100)]))

2
35
63


In [ ]:
questions[21]

'Which  American politician did Donahue replaced '

In [ ]:
supporting_articles[21]

['Sue Donahue is an American politician.  Donahue was appointed in 2016 to serve in the Arizona State Senate representing the fifth legislative district as a member of the Republican Party.  Donahue replaced Kelli Ward who resigned to run for the United States Senate.  Donahue did not run for re-election in 2016 and was replaced by Sonny Borrelli.',
 'Kelli Ward ("née" Kaznoski; born January 25, 1969) is an American politician and osteopathic physician.  A member of the Republican Party, Ward was elected in 2012 to serve in the Arizona State Senate representing the fifth legislative district.  She was unopposed for election in 2014.  While serving in office, she continued to practice medicine in the emergency departments in Lake Havasu City and Kingman, Arizona.']

In [ ]:
retrievals[21]

['Maurice A. Donahue (September 21, 1918 – January 13, 1999) is an American politician who served as President of the Massachusetts Senate from 1964 to 1971.',
 'Sue Donahue is an American politician.  Donahue was appointed in 2016 to serve in the Arizona State Senate representing the fifth legislative district as a member of the Republican Party.  Donahue replaced Kelli Ward who resigned to run for the United States Senate.  Donahue did not run for re-election in 2016 and was replaced by Sonny Borrelli.',
 'Maurice A. Donahue (September 21, 1918 – January 13, 1999) is an American politician who served as President of the Massachusetts Senate from 1964 to 1971.',
 'Frank J. Donahue (1881–1979) was an American politician who served as the Massachusetts Secretary of the Commonwealth, Chairman of the Massachusetts Democratic State Committee, and as an Associate Justice of the Massachusetts Superior Court.']

In [ ]:
hit_per_q[21]

1

In [ ]:
hard_hits = 0
num_hard = 0
for i in range(len(responses)):
  if level[i] == "hard":
    num_hard += 1
    if hit_per_q[i] == 2:
      hard_hits += 1



print(num_hard)
print(hard_hits / num_hard)


16
0.6875


In [ ]:
for i in range(len(responses)):
  if hit_per_q[i] == 2:
    hard_hits += 1

hard_hits/100

0.74

In [ ]:
print(num_hit_rel / 100)
print(num_hit / 200)

0.97
0.79


In [ ]:
print(mrr_sum)

8.5


In [ ]:
ans_cor = []

for i in tqdm(range(len(responses))):
  prompt = create_prompt_check(i)



  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0
  )


  if response.choices[0].text.strip().lower().startswith("yes"):
    ans_cor.append(1)
  else:
    ans_cor.append(0)

100%|██████████| 100/100 [00:38<00:00,  2.56it/s]


In [ ]:
#accuracy of the answers

np.sum(ans_cor) / len(ans_cor)

0.72

In [ ]:
#Exporting this evaluation as a dataframe

eval_df = pd.DataFrame({
    'question': questions,
    'ground_truth': answers,
    'rag_response': responses,
    'level': level,
    'correct_or_not': ans_cor
})

eval_df.to_csv("vanilla_hotpot_eval_refl+rerank.csv")

In [ ]:
questions[92]

'The Swedish-British entertainment studio executive and film executive producer, who was the Executive producer for a 2016 American action thriller film directed by Babak Najafi, currently serve?'

In [ ]:
cors = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}
nums = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}

for i in range(len(ans_cor)):
  nums[level[i]] += 1
  if ans_cor[i] == 1:
    cors[level[i]] += 1


print(cors['easy'] / nums['easy'])
print(cors['medium'] / nums['medium'])
print(cors['hard'] / nums['hard'])

#dont pay attention to this as this has been manually evaluated

0.8518518518518519
0.7368421052631579
0.4375


## Tree-RAG

After the failure of a decomposition based strategy, we will instead try and see if a tree based strategy works instead.

In [ ]:
def next_q_extraction_tree(ques, passage):

  prompt = """
  I know the following information in context:

  -----------
  Context
  {passage}
  -----------

  I need to answer the question: {ques}.
  What more information do I need in addition to the context to answer this?
  Frame your response as a simple, one hop follow-up question.
  """.format(ques = ques, passage = passage)

  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=prompt,
      temperature = 0,
      max_tokens = 100
    )

  return response.choices[0].text.strip()


def construct_final_prompt_tree(ques, passage_1, passage_2, passage_3):

  prompt = """
  You are given the following passages as context

  Passage 1: {passage_1}
  Passage 2: {passage_2}
  Passage 3: {passage_3}

  Based on the information in the context passages below, answer the following question:

  Question: {ques}
  """.format(ques = ques, passage_1 = passage_1, passage_2 = passage_2, \
             passage_3 = passage_3)

  return prompt


In [ ]:
engine.query("What is the capital of India?")

Response(response='The capital of India is New Delhi.', source_nodes=[NodeWithScore(node=TextNode(id_='4b2cb0cf-74ce-4986-a138-69b0f4a18db9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5d51b2b6-e2d6-4629-a7dc-7d93083749bb', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='4aa7c1503f683048d198ebe9d923639511b059ca52d3a7cec4f81f0ec051cb8e'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='be625152-85fd-43e0-926a-b5e252848df6', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='ec91b402d245042ca723bc7dd2c883009d0efbc0f3c18aafc6ec13c8090d8174'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='8d4bd721-db27-463f-9da0-ae527c1b79cd', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='f1cddd244c3ec0fa99d0d7262586cb4a8123c0f4a9838c976e6fa256c712f396')}, text='Nagpur is the winter capital, a sprawling metropolis and the third largest city of the I

In [ ]:
#Evaluation block
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])
similarity_top_k = 4
hit_per_q = []
num_hit = 0
num_hit_rel = 0
responses = []
mrr_sum = 0
engine = index.as_query_engine(similarity_top_k = similarity_top_k,
                               llm = oai(temperature = 0, model = "gpt-3.5-turbo-instruct"))

retrievals = []

for i in tqdm(range(0, 100)):
  retrieved_contexts = []
  print("Iteration " + str(i))
  hit_perc = 0
  q = questions[i]
  hit_flag = False

  # query_result = engine.query(q)
  query_result = engine.query(q)

  hits = [query_result.source_nodes[i].text for i in range(len(query_result.source_nodes))]

  # facts_string = ""

  # for j in range(2):
  #   facts_string += "Fact " + str(j+1) + ": " + hits[j] + "\n"

  # claim = create_claim(q, query_result.response)

  # refl_prompt = create_reflection_prompt(facts_string, query_result.response)

  # reflection = client.completions.create(
  #   model="gpt-3.5-turbo-instruct",
  #   prompt=refl_prompt,
  #   temperature = 0,
  #   max_tokens = 100
  # )

  # print(reflection.choices[0].text.strip())

  # if reflection.choices[0].text.strip().lower().split(' ')[0].startswith("no"):

  print("follow-up question chosen")

  new_contexts = []
  next_questions = []

  for j in range(len(hits[:3])):
    next_qs = next_q_extraction_tree(q, hits[j])
    query_result_new = engine.query(next_qs)
    new_con = [hits[j]] + [query_result_new.source_nodes[k].text for k in range(2)]
    retrieved_contexts.extend(new_con)
    new_contexts.append('. '.join(new_con))
    next_questions.append(next_qs)

  # next_q = next_q_extraction(q, facts_string)

  # print(f"Next question: {next_q}")

  # # query_result_new = engine.query(next_q)
  # query_result_new = engine.query(next_q)

  # new_hits = [query_result_new.source_nodes[i].text for i in range(len(query_result_new.source_nodes))]

  final_prompt = construct_final_prompt_tree(q, passage_1 = new_contexts[0], passage_2 = new_contexts[1], \
                                        passage_3 = new_contexts[2])

  print(final_prompt)
  response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt=final_prompt,
      temperature = 0,
      max_tokens = 100,
      logprobs = 1
    ).choices[0].text.strip()


  print(response)

  responses.append(response)


  for j in range(len(supporting_articles[i])):
    if supporting_articles[i][j] in retrieved_contexts:
      hit_flag = True
      num_hit += 1
      hit_perc += 1

  if hit_flag:
    num_hit_rel += 1

  hit_per_q += [hit_perc]

  0%|          | 0/100 [00:00<?, ?it/s]

Iteration 0
follow-up question chosen

  You are given the following passages as context

  Passage 1: First for Women is a woman's magazine published by Bauer Media Group in the USA.  The magazine was started in 1989.  It is based in Englewood Cliffs, New Jersey.  In 2011 the circulation of the magazine was 1,310,696 copies.. Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.  In May 1846 it was merged into "Godey's Lady's Book".. Arthur's Home Magazine (1852-ca.1898) or Ladies' Home Magazine was an American periodical published in Philadelphia by Timothy Shay Arthur.  Editors Arthur and Virginia Francis Townsend selected writing and illustrations intended to appeal to female readers.  Among the contributors: Mary Tyler Peabody Mann and Kate Sutherland.  In its early years the monthly comprised a selecti

  1%|          | 1/100 [00:20<34:04, 20.65s/it]

Arthur's Magazine was started first.
Iteration 1
follow-up question chosen

  You are given the following passages as context

  Passage 1: The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group.. The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Oberoi Group.. The Oberoi Group is a hotel company with its head office in Delhi.  Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands.
  Passage 2: The Oberoi Group is a hotel company with its head office in Delhi.  Founded in 1934, the company owns and/or operates 30+ luxury hotels and two river cruise ships in six countries, primarily under its Oberoi Hotels & Resorts and Trident Hotels brands.. The Oberoi family is an Indian family that is famous for its involvement in hotels, namely through The Obe

  2%|▏         | 2/100 [00:34<27:12, 16.66s/it]

...

Answer: Senator David Boren
Iteration 98
follow-up question chosen

  You are given the following passages as context

  Passage 1: The Thoen Stone is a sandstone slab dated 1834 that was discovered in the Black Hills of South Dakota by Louis Thoen in 1887.  The discovery of the stone called into question the first discovery of gold and the history of gold mining in the Black Hills; it would mean that gold was discovered in the Black Hills 40 years before the Custer Expedition of 1874 and the subsequent Black Hills Gold Rush.  It is currently on display at the Adams Museum & House in Deadwood, South Dakota.. The Cooper–Frost–Austin House is a historic Colonial American house, built in 1681.  It is located at 21 Linnaean Street, Cambridge, Massachusetts.  It is the oldest extant home in Cambridge and is owned and operated as a non-profit museum by Historic New England.  The house is rarely open for public tours, but private tours can be arranged during the summer months.. The Dey Mansio

 99%|█████████▉| 99/100 [25:05<00:22, 22.66s/it]

The Thoen Stone is on display at the Adams Museum & House in Deadwood, South Dakota, which is located in Lawrence County.
Iteration 99
follow-up question chosen

  You are given the following passages as context

  Passage 1: MedStar Washington Hospital Center is the largest private hospital in Washington, D.C. A member of MedStar Health, the not-for-profit Hospital Center is licensed for 926 beds.  Health services in primary, secondary and tertiary care are offered to adult and neonatal patients.  It also serves as a teaching hospital for Georgetown University School of Medicine.. Children’s National Medical Center (formerly DC Children’s Hospital) is ranked among the top 10 children’s hospitals in the country by "U.S. News & World Report."  Located just north of the McMillan Reservoir and Howard University, it shares grounds with Washington Hospital Center, National Rehabilitation Hospital, and the DC Veterans Affairs Medical Center.  Kurt Newman, M.D., has served as the president an

100%|██████████| 100/100 [25:21<00:00, 15.21s/it]

MedStar Washington Hospital Center is the largest private hospital in Washington, D.C.


In [ ]:
responses

In [ ]:
hard_hits = 0
num_hard = 0
for i in range(len(responses)):
  if level[i] == "hard":
    num_hard += 1
    if hit_per_q[i] == 2:
      hard_hits += 1



print(num_hard)
print(hard_hits / num_hard)


16
0.6875


In [ ]:
hard_hits = 0
for i in range(len(responses)):
  if hit_per_q[i] == 2:
    hard_hits += 1

hard_hits/100

0.69

In [ ]:
print(num_hit_rel / 100)
print(num_hit / 200)

0.98
0.835


In [ ]:
print(mrr_sum)

8.5


In [ ]:
ans_cor = []

for i in tqdm(range(len(responses))):
  prompt = create_prompt_check(i)



  response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt=prompt,
    temperature = 0
  )


  if response.choices[0].text.strip().lower().startswith("yes"):
    ans_cor.append(1)
  else:
    ans_cor.append(0)

100%|██████████| 100/100 [00:37<00:00,  2.65it/s]


In [ ]:
#accuracy of the answers

np.sum(ans_cor) / len(ans_cor)

0.74

In [ ]:
#Exporting this evaluation as a dataframe

eval_df = pd.DataFrame({
    'question': questions,
    'ground_truth': answers,
    'rag_response': responses,
    'level': level,
    'correct_or_not': ans_cor
})

eval_df.to_csv("vanilla_hotpot_eval_tree.csv")

In [ ]:
questions[92]

'The Swedish-British entertainment studio executive and film executive producer, who was the Executive producer for a 2016 American action thriller film directed by Babak Najafi, currently serve?'

In [ ]:
cors = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}
nums = {
    'hard': 0,
    'easy': 0,
    'medium': 0
}

for i in range(len(ans_cor)):
  nums[level[i]] += 1
  if ans_cor[i] == 1:
    cors[level[i]] += 1


print(cors['easy'] / nums['easy'])
print(cors['medium'] / nums['medium'])
print(cors['hard'] / nums['hard'])

#dont pay attention to this as this has been manually evaluated

0.8518518518518519
0.7368421052631579
0.4375


In [ ]:
engine.query(next_questions[2])

Response(response='The narrator of "Frontier" is Walter Darwin Coy and he is not related to the Lone Gunmen from Laredo.', source_nodes=[NodeWithScore(node=TextNode(id_='679b558e-4ddc-49ae-acbd-61118371f02c', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='2b43b0d4-8510-48b9-9476-9462ae1edd42', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='e510add6924597155a7f8e3e580d59c3d30853ba7981693d224c324921e7189d'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='3f464bf5-851e-4583-a782-3b43e9a2173a', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d320baa52eb36a07d59481978be700135cc3457607ecaca894a4658facc0bb6c'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='af2c51cf-6ed4-4ad2-90db-ad647f5d89bf', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='d015c51b39339214807453bd338c56e6941ecf1c601cb58c929205f63a328509')}, text='The Lone Gunmen are a

In [ ]:
response = client.completions.create(
      model="gpt-3.5-turbo-instruct",
      prompt="how are you",
      temperature = 0,
      max_tokens = 100,
      logprobs = 1
    )

response

Completion(id='cmpl-9GGUreUB6uk2ofk9eJpHcv6gBwcRQ', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=Logprobs(text_offset=[11, 13, 14, 16, 22, 27, 28, 34, 38, 42, 49, 50, 54, 60, 64], token_logprobs=[-0.44241193, -1.0345889, -0.8429186, -0.7543398, -0.58498454, -0.44976145, -0.07949553, -0.00022642381, -0.25797638, -0.00015169414, -0.23518401, -0.010540018, -0.20009966, -0.04421936, -0.017421667], tokens=['\n\n', 'I', "'m", ' doing', ' well', ',', ' thank', ' you', ' for', ' asking', '.', ' How', ' about', ' you', '?'], top_logprobs=[{'\n\n': -0.44241193}, {'I': -1.0345889}, {"'m": -0.8429186}, {' doing': -0.7543398}, {' well': -0.58498454}, {',': -0.44976145}, {' thank': -0.07949553}, {' you': -0.00022642381}, {' for': -0.25797638}, {' asking': -0.00015169414}, {'.': -0.23518401}, {' How': -0.010540018}, {' about': -0.20009966}, {' you': -0.04421936}, {'?': -0.017421667}]), text="\n\nI'm doing well, thank you for asking. How about you?")], created=1713663689, model='g

In [ ]:
hit_per_q

[2, 2, 2, 0, 1, 2, 1, 1, 2, 1]